In [199]:
%%capture
%run 03_block.ipynb

We can validate a mined block pow very easily. 

In [200]:
def val_pow(mb):
    mb_b        = bytes(mb)
    candidate   = mb_b + str(mb.nonce).encode()
    candidate_h = sha(candidate)
    if candidate_h[:mb.diff] == '0'*mb.diff: return True
    else                                   : return False
    
assert val_pow(mb)

If we change something in the block the pow should break.

In [201]:
mb.prev_hash = rh()
assert not val_pow(mb)

## Blockchain

Consists of a number of blocks. Each block points to the block that came before it. Imagine a linked list of blocks linked by `prev_hash`.

In [202]:
class Blockchain:
    def __init__(self, genesis):
        self.balances,self.nonces = {},{}
        self.blocks   = []
        self.init(genesis)
        
    def init(self, genesis):
        for tx in genesis.txs: 
            self.balances[tx.to] = tx.value
            self.nonces  [tx.fr] = 1
        self.blocks.append(genesis)

    def candidate(self, txs):
        return Block(txs, self.blocks[-1].hash, len(self.blocks)+1)
        
    def add(self, mb):
        assert self.val(mb)
        for tx in mb.txs: assert self.apply(tx)
        self.apply_fees(mb)
        self.blocks.append(mb)
        return True
        
    def apply(self, tx):
        assert val_sig(tx)
        assert self.val_nonce(tx) 
        assert self.balances[tx.fr] - tx.value > 0
        self.apply_balance(tx)
        self.apply_nonce(tx)
        return True
    
    def apply_balance(self, tx):
        self.balances[tx.fr] -= tx.value
        if tx.to in self.balances: self.balances[tx.to] += tx.value
        else:                      self.balances[tx.to]  = tx.value
            
    def apply_nonce(self, tx):
        if tx.to not in self.nonces: self.nonces[tx.to] = 0
    
    def apply_fees(self, mb):
        for tx in mb.txs: self.balances[mb.miner] += tx.fee
    
    def val_nonce(self, tx):
        if tx.fr in self.nonces:
            assert self.nonces[tx.fr] == tx.nonce
            self.nonces[tx.fr] += 1
        else: 
            assert tx.nonce == 0
            self.nonces[tx.fr] = 0
        return True
    
    def val(self, mb): 
        assert val_pow(mb)
        assert mb.number == len(self.blocks)+1
        if len(self.blocks) > 0: assert self.blocks[-1].hash == mb.prev_hash
        return True
    
    def __str__(self):
        return '\n'.join(f'{ph(k)}\nbalance: {v} \nnonce: {self.nonces[k]}\n' for k,v in self.balances.items())

In [203]:
acc1,acc2,acc3 = Account(),Account(),Account()
miner = Miner(acc3)

In [204]:
tx1 = acc1.signed_tx(acc2, 24, 0.1)
tx2 = acc2.signed_tx(acc1, 18, 0.12)

gb = miner.mine_genesis([tx1, tx2], reward=200)

In [205]:
bc = Blockchain(gb); print(bc)

🔘 0xB211D4eb93d484...8DB
balance: 200 
nonce: 1

🕝 0xF7a3AdE815b1ff...88F
balance: 24 
nonce: 1

💎 0x2843561fA7FCDa...eD2
balance: 18 
nonce: 1



In [206]:
tx1 = acc1.signed_tx(acc2, 3, 0.05)
tx2 = acc2.signed_tx(acc1, 9, 0.16)

block = bc.candidate([tx1,tx2])
mb    = miner.mine(block, 2, 100)
bc.add(mb); print(bc)

🔘 0xB211D4eb93d484...8DB
balance: 200.21 
nonce: 2

🕝 0xF7a3AdE815b1ff...88F
balance: 18 
nonce: 2

💎 0x2843561fA7FCDa...eD2
balance: 24 
nonce: 2



In [207]:
acc4 = Account()
tx1 = acc2.signed_tx(acc4, 12, 1.5)

block = bc.candidate([tx1])
mb    = miner.mine(block, 2, 100)
bc.add(mb); print(bc)

🔘 0xB211D4eb93d484...8DB
balance: 201.71 
nonce: 3

🕝 0xF7a3AdE815b1ff...88F
balance: 6 
nonce: 3

💎 0x2843561fA7FCDa...eD2
balance: 24 
nonce: 2

💾 0x58c83B7Bb27cd7...A79
balance: 12 
nonce: 0

